In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances_chunked
import trashpandas as tp

In [2]:
df = pd.read_csv('data/year_prediction.csv')
df

,label,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance69,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515340,2006,51.28467,45.88068,22.19582,-5.53319,-3.61835,-16.36914,2.12652,5.18160,-8.66890,...,4.81440,-3.75991,-30.92584,26.33968,-5.03390,21.86037,-142.29410,3.42901,-41.14721,-15.46052
515341,2006,49.87870,37.93125,18.65987,-3.63581,-27.75665,-18.52988,7.76108,3.56109,-2.50351,...,32.38589,-32.75535,-61.05473,56.65182,15.29965,95.88193,-10.63242,12.96552,92.11633,10.88815
515342,2006,45.12852,12.65758,-38.72018,8.80882,-29.29985,-2.28706,-18.40424,-22.28726,-4.52429,...,-18.73598,-71.15954,-123.98443,121.26989,10.89629,34.62409,-248.61020,-6.07171,53.96319,-8.09364
515343,2006,44.16614,32.38368,-3.34971,-2.49165,-19.59278,-18.67098,8.78428,4.02039,-12.01230,...,67.16763,282.77624,-4.63677,144.00125,21.62652,-29.72432,71.47198,20.32240,14.83107,39.74909


In [3]:
def convert_array_to_ints(a: np.array, n=2) -> int:
    a *= (n * 10)
    return a.astype(int)

In [83]:
generator = pairwise_distances_chunked(df.drop('label', axis=1), n_jobs=8, )

In [84]:
class DataFrameChunks:
    def __init__(self, generator):
        self.generator = generator
        self.n = 0
        
    def __next__(self) -> pd.DataFrame:
        chunk = next(generator)
        new_n = self.n + len(chunk)-1
        index = range(self.n, new_n+1)
        self.n = new_n + 1
        return pd.DataFrame(chunk, index=index)
    
    def __iter__(self):
        return self

In [85]:
df_generator = DataFrameChunks(generator)

In [55]:
#next(df_generator)

In [78]:
def top_scores(row, row_date, dates, n=10):
    df = pd.DataFrame({
        'x_index': row.name,
        'score': row,
        'y_date':  dates
    })
    df = df[df['y_date'] < row_date]
    df.index.name = 'y_index'
    return df.sort_values('score').head(n).drop('y_date', axis=1).reset_index()

top_scores(row0, df.iloc[0]['label'], df['label'])

,y_index,x_index,score
0,295150,0,416.737810
1,432002,0,432.787992
2,5426,0,450.573031
3,69529,0,452.135393
4,410304,0,476.283160
5,182997,0,480.559777
6,381923,0,481.647090
7,288080,0,482.275757
8,372565,0,487.443584
9,133144,0,488.176847


In [ ]:
for i, row in df0.iterrows():
    scores = top_scores(row, df0.iloc[i]['label'])

In [ ]:
df.apply(top_scores)

In [86]:
results = pd.DataFrame({'y_index': [], 'x_index': [], 'score': []})
for chunk_i, chunk in enumerate(df_generator):
    for row_i, row in chunk.iterrows():
        scores = top_scores(row, df.iloc[row_i]['label'], df['label'])
        results = pd.concat([results, scores])
    print(row_i, 'done', results.memory_usage(deep=True).sum(), 'memory used.')
results

259 done 83200 memory used.
519 done 166400 memory used.
779 done 249600 memory used.
1039 done 332800 memory used.
1299 done 416000 memory used.
1559 done 499200 memory used.
1819 done 582400 memory used.
2079 done 665600 memory used.
2339 done 748800 memory used.
2599 done 832000 memory used.
2859 done 915200 memory used.
3119 done 998400 memory used.
3379 done 1081600 memory used.
3639 done 1164800 memory used.
3899 done 1248000 memory used.
4159 done 1331200 memory used.
4419 done 1414400 memory used.
4679 done 1497600 memory used.
4939 done 1580800 memory used.
5199 done 1664000 memory used.
5459 done 1747200 memory used.
5719 done 1830400 memory used.
5979 done 1913600 memory used.
6239 done 1996800 memory used.
6499 done 2080000 memory used.
6759 done 2163200 memory used.
7019 done 2246400 memory used.
7279 done 2329600 memory used.
7539 done 2412800 memory used.
7799 done 2496000 memory used.
8059 done 2579200 memory used.
8319 done 2662400 memory used.
8579 done 2745600 memory

66299 done 21214400 memory used.
66559 done 21297600 memory used.
66819 done 21380800 memory used.
67079 done 21464000 memory used.
67339 done 21547200 memory used.
67599 done 21630400 memory used.
67859 done 21713600 memory used.
68119 done 21796800 memory used.
68379 done 21880000 memory used.
68639 done 21963200 memory used.
68899 done 22046400 memory used.
69159 done 22129600 memory used.
69419 done 22212800 memory used.
69679 done 22296000 memory used.
69939 done 22379200 memory used.
70199 done 22462400 memory used.
70459 done 22545600 memory used.
70719 done 22628800 memory used.
70979 done 22712000 memory used.
71239 done 22795200 memory used.
71499 done 22878400 memory used.
71759 done 22961600 memory used.
72019 done 23044800 memory used.
72279 done 23128000 memory used.
72539 done 23211200 memory used.
72799 done 23294400 memory used.
73059 done 23377600 memory used.
73319 done 23460800 memory used.
73579 done 23544000 memory used.
73839 done 23627200 memory used.
74099 done

129999 done 41598400 memory used.
130259 done 41681600 memory used.
130519 done 41764800 memory used.
130779 done 41848000 memory used.
131039 done 41931200 memory used.
131299 done 42014400 memory used.
131559 done 42097600 memory used.
131819 done 42180800 memory used.
132079 done 42264000 memory used.
132339 done 42347200 memory used.
132599 done 42430400 memory used.
132859 done 42513600 memory used.
133119 done 42596800 memory used.
133379 done 42680000 memory used.
133639 done 42763200 memory used.
133899 done 42846400 memory used.
134159 done 42929600 memory used.
134419 done 43012800 memory used.
134679 done 43096000 memory used.
134939 done 43179200 memory used.
135199 done 43262400 memory used.
135459 done 43345600 memory used.
135719 done 43428800 memory used.
135979 done 43512000 memory used.
136239 done 43595200 memory used.
136499 done 43678400 memory used.
136759 done 43761600 memory used.
137019 done 43844800 memory used.
137279 done 43928000 memory used.
137539 done 44

192659 done 61649600 memory used.
192919 done 61732800 memory used.
193179 done 61816000 memory used.
193439 done 61899200 memory used.
193699 done 61982400 memory used.
193959 done 62065600 memory used.
194219 done 62148800 memory used.
194479 done 62232000 memory used.
194739 done 62315200 memory used.
194999 done 62398400 memory used.
195259 done 62481600 memory used.
195519 done 62564800 memory used.
195779 done 62648000 memory used.
196039 done 62731200 memory used.
196299 done 62814400 memory used.
196559 done 62897600 memory used.
196819 done 62980800 memory used.
197079 done 63064000 memory used.
197339 done 63147200 memory used.
197599 done 63230400 memory used.
197859 done 63313600 memory used.
198119 done 63396800 memory used.
198379 done 63480000 memory used.
198639 done 63563200 memory used.
198899 done 63645888 memory used.
199159 done 63729088 memory used.
199419 done 63812288 memory used.
199679 done 63895488 memory used.
199939 done 63978688 memory used.
200199 done 64

255319 done 81700160 memory used.
255579 done 81783360 memory used.
255839 done 81866560 memory used.
256099 done 81949760 memory used.
256359 done 82032960 memory used.
256619 done 82116160 memory used.
256879 done 82199360 memory used.
257139 done 82282560 memory used.
257399 done 82365760 memory used.
257659 done 82448960 memory used.
257919 done 82532160 memory used.
258179 done 82615360 memory used.
258439 done 82698560 memory used.
258699 done 82781760 memory used.
258959 done 82864960 memory used.
259219 done 82948160 memory used.
259479 done 83031360 memory used.
259739 done 83114560 memory used.
259999 done 83197760 memory used.
260259 done 83280960 memory used.
260519 done 83364160 memory used.
260779 done 83447360 memory used.
261039 done 83530560 memory used.
261299 done 83613760 memory used.
261559 done 83696960 memory used.
261819 done 83780160 memory used.
262079 done 83863360 memory used.
262339 done 83946560 memory used.
262599 done 84029760 memory used.
262859 done 84

317979 done 101751040 memory used.
318239 done 101834240 memory used.
318499 done 101917440 memory used.
318759 done 102000640 memory used.
319019 done 102083840 memory used.
319279 done 102167040 memory used.
319539 done 102250240 memory used.
319799 done 102333440 memory used.
320059 done 102416640 memory used.
320319 done 102499840 memory used.
320579 done 102583040 memory used.
320839 done 102666240 memory used.
321099 done 102749440 memory used.
321359 done 102832640 memory used.
321619 done 102915840 memory used.
321879 done 102999040 memory used.
322139 done 103082240 memory used.
322399 done 103165440 memory used.
322659 done 103248640 memory used.
322919 done 103331840 memory used.
323179 done 103415040 memory used.
323439 done 103498240 memory used.
323699 done 103581440 memory used.
323959 done 103664640 memory used.
324219 done 103747840 memory used.
324479 done 103831040 memory used.
324739 done 103914240 memory used.
324999 done 103997440 memory used.
325259 done 10408064

379079 done 121303040 memory used.
379339 done 121386240 memory used.
379599 done 121469440 memory used.
379859 done 121552640 memory used.
380119 done 121635840 memory used.
380379 done 121719040 memory used.
380639 done 121802240 memory used.
380899 done 121885440 memory used.
381159 done 121968640 memory used.
381419 done 122051840 memory used.
381679 done 122135040 memory used.
381939 done 122218240 memory used.
382199 done 122301440 memory used.
382459 done 122384640 memory used.
382719 done 122467840 memory used.
382979 done 122551040 memory used.
383239 done 122634240 memory used.
383499 done 122717440 memory used.
383759 done 122800640 memory used.
384019 done 122883840 memory used.
384279 done 122967040 memory used.
384539 done 123050240 memory used.
384799 done 123133440 memory used.
385059 done 123216640 memory used.
385319 done 123299840 memory used.
385579 done 123383040 memory used.
385839 done 123466240 memory used.
386099 done 123549440 memory used.
386359 done 12363264

440179 done 140855040 memory used.
440439 done 140938240 memory used.
440699 done 141021440 memory used.
440959 done 141104640 memory used.
441219 done 141187840 memory used.
441479 done 141271040 memory used.
441739 done 141354240 memory used.
441999 done 141437440 memory used.
442259 done 141520640 memory used.
442519 done 141603840 memory used.
442779 done 141687040 memory used.
443039 done 141770240 memory used.
443299 done 141853440 memory used.
443559 done 141936640 memory used.
443819 done 142019840 memory used.
444079 done 142103040 memory used.
444339 done 142186240 memory used.
444599 done 142269440 memory used.
444859 done 142352640 memory used.
445119 done 142435840 memory used.
445379 done 142519040 memory used.
445639 done 142602240 memory used.
445899 done 142685440 memory used.
446159 done 142768640 memory used.
446419 done 142851840 memory used.
446679 done 142935040 memory used.
446939 done 143018240 memory used.
447199 done 143101440 memory used.
447459 done 14318464

501279 done 160407040 memory used.
501539 done 160490240 memory used.
501799 done 160573440 memory used.
502059 done 160656640 memory used.
502319 done 160739840 memory used.
502579 done 160823040 memory used.
502839 done 160906240 memory used.
503099 done 160989440 memory used.
503359 done 161072640 memory used.
503619 done 161155840 memory used.
503879 done 161239040 memory used.
504139 done 161322240 memory used.
504399 done 161405440 memory used.
504659 done 161488640 memory used.
504919 done 161571840 memory used.
505179 done 161655040 memory used.
505439 done 161738240 memory used.
505699 done 161821440 memory used.
505959 done 161904640 memory used.
506219 done 161987840 memory used.
506479 done 162071040 memory used.
506739 done 162154240 memory used.
506999 done 162237440 memory used.
507259 done 162320640 memory used.
507519 done 162403840 memory used.
507779 done 162487040 memory used.
508039 done 162570240 memory used.
508299 done 162653440 memory used.
508559 done 16273664

,y_index,x_index,score
0,295150.0,0.0,416.737810
1,432002.0,0.0,432.787992
2,5426.0,0.0,450.573031
3,69529.0,0.0,452.135393
4,410304.0,0.0,476.283160
...,...,...,...
5,202130.0,515344.0,686.081035
6,187619.0,515344.0,689.758469
7,341249.0,515344.0,693.846842
8,309609.0,515344.0,702.730367


In [87]:
results.to_csv('data/top_10.csv')